## TO DO

2018-11-07

* Should draw the numbers at the correct positions and orientations within the TPS.
* Then burn those numbers in to the dose distribution as a verification of the orientation.

* Rescan maxphan
* Replan
* Remeaure film
* Rescan film
* Compare doses and spatial accuracy

In [ ]:
import os
from glob import glob

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from scipy.interpolate import RegularGridInterpolator

import imageio

import pydicom

from pymedphys.dicom import (
    load_xyz_from_dicom, load_dose_from_dicom,
    get_structure_aligned_cube,
    pull_structure
)

from pymedphys.film import get_interpolated_dose
from pymedphys.geometry import plot_cube, cubify_cube_definition

from pymedphys.plt import pcolormesh_grid

In [ ]:
data_dir = r'S:\Physics\Programming\data\FilmProcessing\SRS\experimentation'

In [ ]:
save_path_2_4 = os.path.join(data_dir, 'dose_2_4.png')
save_path_1_3 = os.path.join(data_dir, 'dose_1_3.png')

In [ ]:
glob(os.path.join(data_dir, '*'))

In [ ]:
glob(os.path.join(data_dir, '*.dcm'))

In [ ]:
dose_file = 'ZZ654329_MS_Dose.dcm'
dose_path = os.path.join(data_dir, dose_file)

In [ ]:
struct_file = 'ZZ654329_StrctrSets.dcm'
struct_path = os.path.join(data_dir, struct_file)

In [ ]:
dcm_dose = pydicom.dcmread(dose_path, force=True)
dcm_struct = pydicom.dcmread(struct_path, force=True)

In [ ]:
dose = load_dose_from_dicom(dcm_dose)
x_dose, y_dose, z_dose = load_xyz_from_dicom(dcm_dose)

In [ ]:
structure_name = 'ANT Box'

In [ ]:
box_x, box_y, box_z = pull_structure(structure_name, dcm_struct)

In [ ]:
min_x = np.min(np.concatenate(box_x))
max_x = np.max(np.concatenate(box_x))

min_y = np.min(np.concatenate(box_y))
max_y = np.max(np.concatenate(box_y))

min_z = np.min(np.concatenate(box_z))
max_z = np.max(np.concatenate(box_z))

In [ ]:
xx=np.concatenate(box_x)
# print(xx)

In [ ]:
x0 = np.array([
    (min_y, min_x, min_z),
    (min_y, max_x, min_z), 
    (max_y, min_x, max_z)
])

cube_definition = cubify_cube_definition(x0)
cube = cubify_cube_definition(cube_definition)

ax = plot_cube(cube)

for x, y, z in zip(box_x, box_y, box_z):
    ax.plot(x, y, z, 'b-')

In [ ]:
cube_definition_array, vectors = get_structure_aligned_cube(x0, structure_name, dcm_struct)

In [ ]:
cube_definition_array

In [ ]:
ax = plot_cube(cube_definition_array)

for x, y, z in zip(box_x, box_y, box_z):
    ax.plot(x, y, z, 'b-')

In [ ]:
ax = plot_cube(cube_definition_array)
colour = iter(['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'])

structures = ['Number1', 'Number2', 'Number3', 'Number4']
for structure_name in structures:
    this_colour = next(colour)
    struct_x, struct_y, struct_z = pull_structure(structure_name, dcm_struct)
    for x, y, z in zip(struct_x, struct_y, struct_z):
        ax.plot(x, y, z, c=this_colour)

# ax.legend()

In [ ]:
dose

In [ ]:
vertices_1_3 = (
    0.5 * vectors[0] + cube_definition_array[0],
    0.5 * vectors[0] + vectors[1] + cube_definition_array[0],
    0.5 * vectors[0] + vectors[1] + vectors[2] + cube_definition_array[0],
    0.5 * vectors[0] + vectors[2] + cube_definition_array[0],
)

vertices_2_4 = (
    0.5 * vectors[1] + cube_definition_array[0],
    0.5 * vectors[1] + vectors[2] + cube_definition_array[0],
    0.5 * vectors[1] + vectors[2] + vectors[0] + cube_definition_array[0],
    0.5 * vectors[1] + vectors[0] + cube_definition_array[0],
)

In [ ]:
vertices_1_3

In [ ]:
side_length = np.sqrt(np.sum(vectors[0]**2))

film_side_length = 56.73  # measured manually on the film image
resolution = 0.1
number_of_points = int(film_side_length / resolution + 1)
margin = (side_length - film_side_length) / 2

margin_ratio = margin / side_length
margin_ratio

In [ ]:
coords_grid_1_3 = np.array([
    [
        vertices_1_3[0] + vectors[1] * step_i + vectors[2] * step_j
        for step_i in np.linspace(1 - margin_ratio, margin_ratio, number_of_points)
    ]
    for step_j in np.linspace( 1 - margin_ratio, margin_ratio, number_of_points)
])

coords_grid_2_4 = np.array([
    [
        vertices_2_4[0] + vectors[2] * step_i + vectors[0] * step_j
        for step_i in np.linspace(margin_ratio, 1 - margin_ratio, number_of_points)
    ]
    for step_j in np.linspace(margin_ratio, 1 - margin_ratio, number_of_points)
])

In [ ]:
dose_ij_indexing = np.swapaxes(dose, 0, 1)

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.RegularGridInterpolator.html
dose_interpolation = RegularGridInterpolator(
    (x_dose, y_dose, z_dose), dose_ij_indexing)

In [ ]:
dose_1_3 = get_interpolated_dose(coords_grid_1_3, dose_interpolation)
dose_2_4 = get_interpolated_dose(coords_grid_2_4, dose_interpolation)

In [ ]:
# Burn orientation and scale markers into the image

dose_2_4[0:11,0:41] = 0
dose_2_4[0:10,0:40] = 10

dose_2_4[-41::,-11::] = 10
dose_2_4[-40::,-10::] = 0


dose_1_3[0:11,0:41] = 0
dose_1_3[0:10,0:40] = 10

dose_1_3[-41::,-11::] = 10
dose_1_3[-40::,-10::] = 0

In [ ]:
film_edge_vector = coords_grid_1_3[0,-1,:] - coords_grid_1_3[-1,-1,:]
calced_film_edge = np.sqrt(np.sum(film_edge_vector**2))
calced_film_edge

In [ ]:
dim = np.shape(coords_grid_1_3)

film_x = np.linspace(0, calced_film_edge, dim[0])
film_y = np.linspace(0, calced_film_edge, dim[1])

x, y = pcolormesh_grid(film_x, film_y, resolution)

plt.figure(figsize=(12,10))
plt.pcolormesh(x, y, np.swapaxes(dose_1_3, 0, 1))
plt.gca().invert_yaxis()
plt.colorbar()

plt.axis('equal')

In [ ]:
dim = np.shape(coords_grid_2_4)

film_x = np.linspace(0, calced_film_edge, dim[0])
film_y = np.linspace(0, calced_film_edge, dim[1])

x, y = pcolormesh_grid(film_x, film_y, resolution)

plt.figure(figsize=(12,10))
plt.pcolormesh(x, y, np.swapaxes(dose_2_4, 0, 1))
plt.gca().invert_yaxis()
plt.colorbar()

plt.axis('equal')

In [ ]:
# imageio.imwrite(save_path_2_4, np.swapaxes(dose_2_4, 0, 1))

In [ ]:
# imageio.imwrite(save_path_1_3, np.swapaxes(dose_1_3, 0, 1))